基本的数据筛选，拿完整数据筛选几轮

In [1]:
import pandas as pd
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
from datetime import datetime
import json
import numpy as np
import cnfundutils
import libtrdb2

cnfundutils.init()

FUNDRESULTS_PATH = './calcfundfull.csv'
dfFundResults = cnfundutils.loadFundResult(FUNDRESULTS_PATH)

FUNDS_PATH = './funds.xlsx'
dfFunds = cnfundutils.loadFundBasicInfo(FUNDS_PATH)

接下来对基金经理做了数据提取

In [5]:
dataManagers = cnfundutils.getManagers(dfFunds)
dfManagers = pd.DataFrame(dataManagers)

In [6]:
dfManagers.sort_values(by='name')

,name,sex,education,country,resume,startDate,endDate,fundcode,fundname,company,maxdrawdown,sharpe,annualizedreturns,annualizedvolatility,totalreturns,maxdrawdown3m,sharpe3m,annualizedreturns3m,annualizedvolatility3m,totalreturns3m,workdays
7341,DANIEL DONGNING SUN,True,博士后,美国,"北京大学硕士,美国哥伦比亚大学博士,...",20180123,20200930,005085,平安量化先锋C,平安基金管理有限公司,0.09876,0.05940,0.04699,0.28607,0.04699,0.09876,0.94212,0.25750,0.24148,0.25750,888
7367,DANIEL DONGNING SUN,True,博士后,美国,"北京大学硕士,美国哥伦比亚大学博士,约翰霍普金斯大学博士后。先后担任瑞士再保险自营交易部量化...",20171226,20200930,005113,平安沪深300指数量化增强A,平安基金管理有限公司,0.30693,0.30622,0.09248,0.20404,0.25540,0.22885,0.54999,0.14421,0.20765,0.36308,1042
7368,DANIEL DONGNING SUN,True,博士后,美国,"北京大学硕士,美国哥伦比亚大学博士,约翰霍普金斯大学博士后。先后担任瑞士再保险自营交易部量化...",20171226,20200930,005114,平安沪深300指数量化增强C,平安基金管理有限公司,0.31244,0.26568,0.08426,0.20424,0.23270,0.23404,0.50568,0.13511,0.20786,0.34018,1042
16313,DANIEL DONGNING SUN,True,博士后,美国,"北京大学硕士,美国哥伦比亚大学博士,...",20170905,20200930,700002,平安深证300指数增强,平安基金管理有限公司,0.07588,0.28576,0.10340,0.25687,0.10340,0.07588,1.51122,0.35922,0.21785,0.35922,1028
16095,Denis Zhang,True,硕士,中国,"Denis Zhang(张惟)先生,CFA,FRM,金融...",20111129,20200930,660011,农银中证500指数,农银汇理基金管理有限公司,0.64741,0.12807,0.06414,0.26654,0.42559,0.64741,0.21151,0.08596,0.26457,0.59159,804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13456,龚炜,True,硕士,中国,"龚炜先生,安徽财经大学金融学硕士,研...",20110804,20200930,350008,天治新消费混合,天治基金管理有限公司,0.57474,0.49370,0.16217,0.26772,1.30939,0.57474,0.40820,0.13899,0.26700,1.15686,393
13857,龚炜,True,硕士,中国,"龚炜先生,安徽财经大学金融学硕士,研...",20170912,20200930,410009,华富量子生命力混合,华富基金管理有限公司,0.15298,0.22645,0.08549,0.24504,0.08549,0.18330,1.03015,0.29284,0.25515,0.29284,898
13825,龚炜,True,硕士,中国,"龚炜先生,安徽财经大学金融学硕士,研...",20121221,20200930,410003,华富成长趋势混合,华富基金管理有限公司,0.13199,2.29667,0.53911,0.22167,0.64989,0.13199,1.59189,0.38506,0.22304,0.55912,2399
5113,龚炜,True,硕士,中国,"龚炜先生,安徽财经大学金融学硕士,研...",20170509,20200930,003152,华富天鑫灵活配置混合A,华富基金管理有限公司,0.12549,0.71600,0.21750,0.26187,0.21750,0.13738,2.11532,0.58334,0.26159,0.58334,1024


一共有多少个基金经理呢？

In [7]:
dfManagers['name'].nunique()

3173

我们来看看哪些基金经理跳过槽，也就是待过2家以上基金公司的

In [8]:
dfmgrcompany = dfManagers.groupby(['name'])['company'].nunique().reset_index()
dfmgrcompany.loc[dfmgrcompany['company'] > 1].sort_values(by='company')

,name,company
11,RU PING,2
1985,王涛,2
1982,王海涛,2
1978,王海,2
1970,王永宏,2
...,...,...
711,孙伟,4
1139,徐荔蓉,4
382,刘辉,5
2005,王磊,5


可以再来看看这些基金经理管理过的多个基金数量

In [9]:
dfmgrfund = dfManagers.groupby(['name'])['fundcode'].nunique().reset_index()
dfmgrfund.loc[dfmgrfund['fundcode'] > 1].sort_values(by='fundcode')

,name,fundcode
1905,王宏远,2
764,孟阳,2
769,季慧娟,2
771,季新星,2
1841,牛志冬,2
...,...,...
3127,黄海峰,41
2028,王超,42
2805,陈凯杨,42
330,刘杰,44


任期平均年化的排行

In [10]:
dfManagers.groupby(['name'])['annualizedreturns'].mean().reset_index().sort_values(by='annualizedreturns', ascending=False)[:100]

,name,annualizedreturns
1921,王建强,3.18933
542,吴超,1.02109
2065,田彧龙,0.93561
2082,皮劲松,0.90947
558,周克平,0.87976
2772,陆彬,0.86900
339,刘武,0.84636
1715,樊勇,0.83103
1711,楼慧源,0.81638
2668,郑有为,0.81417


In [11]:
dfManagersGroup0 = cnfundutils.procManagersWorkDays(dfManagers)
dfManagersGroup0.sort_values(by='annualizedreturns', ascending=False)[:100]

,name,annualizedreturns,fundcode,company,workdays,totalreturns,startDate,endDate,workdays0
1921,王建强,3.18933,14,2,14604,59.64735,20090515,20200930,4156
542,吴超,1.02109,1,1,587,1.50226,20170901,20200930,1125
2065,田彧龙,0.93561,1,1,538,1.29447,20190514,20200930,505
2082,皮劲松,0.90947,2,1,1466,3.48837,20181031,20200930,700
558,周克平,0.87976,1,1,648,1.48234,20190124,20200930,615
2772,陆彬,0.86900,2,1,977,2.17333,20190518,20200930,501
339,刘武,0.84636,1,1,691,1.52577,20181212,20200930,658
1715,樊勇,0.83103,1,1,752,1.63702,20181012,20200930,719
1711,楼慧源,0.81638,1,1,782,1.67525,20180912,20200930,749
2668,郑有为,0.81417,1,1,507,1.05731,20190614,20200930,474


这些是没有数据的记录